<a href="https://colab.research.google.com/github/Samir-atra/Expenses_tracker/blob/main/project/utils/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports

import tensorflow as tf
import pathlib
import IPython
import numpy as np
import keras_tuner


2023-11-03 01:29:00.494699: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-03 01:29:00.525619: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 01:29:00.525646: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 01:29:00.525666: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 01:29:00.531199: I tensorflow/core/platform/cpu_feature_g

Using TensorFlow backend


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# load the data

data_dir = pathlib.Path('/home/samer/Desktop/Tobacco3482-jpg/')


dataset_path, val_data = tf.keras.utils.image_dataset_from_directory(        # Training and validation dataset
    data_dir,
    labels= 'inferred',
    validation_split=0.3,
    subset='both',
    seed= 100,
    batch_size=12,
    image_size=(300,300),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

# dataset_path_test = tf.keras.utils.image_dataset_from_directory(      # Test dataset
#     data_path,
#     labels= 'inferred',
#     seed= 2,
#     batch_size=5,
#     image_size=(299, 299),
#     color_mode="rgb",
#     shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

Found 3102 files belonging to 6 classes.
Using 2172 files for training.
Using 930 files for validation.


2023-11-03 01:29:02.677434: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-03 01:29:02.683406: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-03 01:29:02.683541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# download the transfer learning pre-trained model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (300, 300, 3),
    include_top = False,
    weights = "imagenet",
    
)

In [5]:
# save the pre-trained model
saving_path = pathlib.Path('/home/samer/Desktop/Beedoo/Expenses_tracker_stuff/expenses_tracker_classifier95%test.h5')

base_model.save(saving_path)

/home/samer/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
# load the pre-trained model, build the training model and train it.

base_model = tf.keras.models.load_model(saving_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1)
])

# def build_model(hp):

inputs = tf.keras.Input(shape=(300, 300, 3))
x = augmentation(inputs)
x = rescaling(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation= "relu")(x)
outputs = tf.keras.layers.Dense(6, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)

#     # learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0015),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

    # return model 

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=5,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/home/samer/Desktop/Beedoo/Expenses_tracker/project/Images/",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=5, validation_data = val_data)

# tuner.results_summary()

# # Model checkpoints for longer times of training
# checkpoint_filepath = '/home/samer/Desktop/Beedoo/Expenses_tracker_stuff/checkpoint/'
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=True,
#     monitor='accuracy',
#     mode='max',
#     save_best_only=True)

model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=40,
    # callbacks=[model_checkpoint_callback],
    validation_data = val_data)

# model.summary()

# model.load_weights(checkpoint_filepath)
# model.evaluate(
#     x=x_test,
#     y=y_test,)

# full_model_saving_path = pathlib.Path('/home/samer/Desktop/Beedoo/Expenses_tracker_stuff/expenses_tracker_model_augmented,validated.h5')

# tf.keras.models.save_model(model,                                     # saving the fully trained model
#                            full_model_sadving_path,
#                            overwrite=True,
#                            save_format='tf'
#                            )

Epoch 1/40


2023-11-03 01:29:08.188034: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-03 01:29:11.170224: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.07GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-03 01:29:13.078862: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.91GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-03 01:29:13.078915: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.67GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

138/181 [=====================>........] - ETA: 11s - loss: 1.3182 - accuracy: 0.5839

2023-11-03 01:29:54.082488: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


181/181 [==============================] - ETA: 0s - loss: 1.2420 - accuracy: 0.5994

...

181/181 [==============================] - 68s 375ms/step - loss: 0.2007 - accuracy: 0.9420 - val_loss: 0.3299 - val_accuracy: 0.8710
Epoch 40/40
138/181 [=====================>........] - ETA: 12s - loss: 0.1879 - accuracy: 0.9432

2023-11-03 02:12:11.061544: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


181/181 [==============================] - ETA: 0s - loss: 0.1976 - accuracy: 0.9434

2023-11-03 02:12:29.218617: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


181/181 [==============================] - 68s 372ms/step - loss: 0.1976 - accuracy: 0.9434 - val_loss: 0.3293 - val_accuracy: 0.8720


when working only on 6 classes with the highest count of images, the model gives 94 accuracy and 87 val accuracy and it is still going but that is it for 40 epochs